# Phase 3 – Modélisation & Enrichissement ML

Objectif : enrichir le dataset nettoyé (Phase 2) avec
- **Topics NMF** pour regrouper les offres par thématiques
- **Classe salaire** (haut/bas) pour l’analyse et le dashboard
- **Jeu de données enrichi** prêt pour Phase 4

## Étape 1 – Importer les bibliothèques

- pandas / numpy
- scikit-learn (TF-IDF, NMF, LogisticRegression, métriques)
- utils (Path)
- tqdm pour le confort d’affichage
- Warnings filtrés pour un notebook plus lisible

## Étape 2 – Charger le dataset nettoyé (Phase 2)

On part du fichier produit en Phase 2 : `data/processed/hellowork_cleaned.csv`.

In [1]:
# --- Imports ---
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

# Helpers for cross-cell analysis (Pylance)
df: pd.DataFrame | None = None
X_text = None
clf: LogisticRegression | None = None
X = None
median_salary = 0.0
y = None

print("✅ Imports OK")


✅ Imports OK


## Étape 3 – Harmoniser les colonnes utiles

On crée/normalise les colonnes attendues pour le ML :
- `description_clean` (texte prêt pour TF-IDF)
- `salary_monthly` (valeur numérique mensuelle)
- colonnes catégorielles encodées si présentes (`sector_enc`, `location_enc`, etc.)

In [2]:
# --- Charger le dataset nettoyé ---
clean_path = Path("data/processed/hellowork_cleaned.csv")
assert clean_path.exists(), "Le fichier nettoyé de la Phase 2 est introuvable."

df = pd.read_csv(clean_path, encoding="utf-8")
print(f"📊 Shape: {df.shape}")
print(f"📋 Colonnes: {df.columns.tolist()}")

df.head(3)

📊 Shape: (1219, 14)
📋 Colonnes: ['Sector', 'Job_Title', 'Company', 'Location', 'Contract', 'Salary', 'Description', 'Publication_Date', 'URL', 'Top_Keywords', 'Sector_enc', 'Location_enc', 'Contract_enc', 'Company_enc']


,Sector,Job_Title,Company,Location,Contract,Salary,Description,Publication_Date,URL,Top_Keywords,Sector_enc,Location_enc,Contract_enc,Company_enc
0,Agriculture • Pêche,Alternance - Chargé·e de Formation H/F,Remy Cointreau,Paris - 75,Alternance,"486,49 - 1 801,80 € / mois",Nous recherchons un·e candidat·e : Alternance...,NaN,https://www.hellowork.com/fr-fr/emplois/642118...,"formation,formations,des,de,groupe,aider,créat...",0,0,0,0
1,BTP,Alternance-Gestionnaire Paie H/F,Lafarge France,Issy-les-Moulineaux - 92,Alternance,"486,49 - 1 801,80 € / mois",Pourquoi nous rejoindre ? > Participer à la t...,NaN,https://www.hellowork.com/fr-fr/emplois/729761...,"paie,de,et,la,des,groupe,processus,ses",1,1,0,1
2,BTP,Ouvrier Polyvalent en Menuiserie H/F,Groupe Actual,Auterive - 31,Intérim,"Estimation → 12,36 - 13,50 € / heure",Nous recherchons un(e) menuisier(e) expériment...,NaN,https://www.hellowork.com/fr-fr/emplois/735245...,"recherchons,ayant,un,nous,avons,connaissance,c...",1,2,1,2


## Étape 4 – Créer la cible salaire (High vs Low)

On utilise la médiane du salaire mensuel pour définir `high_salary` (1 = au-dessus de la médiane).

In [3]:
# --- Harmoniser les noms de colonnes ---
# On copie pour éviter les SettingWithCopyWarnings

col_map = {
    "Description_Clean": "description_clean",
    "Salary_Monthly": "salary_monthly",
    "Sector_enc": "sector_enc",
    "Location_enc": "location_enc",
    "Contract_enc": "contract_enc",
    "Company_enc": "company_enc",
}

df = df.rename(columns={k: v for k, v in col_map.items() if k in df.columns})

# Garantir les colonnes minimales
if "description_clean" not in df.columns:
    df["description_clean"] = df.get("Description", "").fillna("").astype(str)
if "salary_monthly" not in df.columns:
    # fallback: utiliser Salary si présent
    df["salary_monthly"] = df.get("Salary", np.nan)

print(df[[c for c in df.columns if "description" in c.lower() or "salary" in c.lower()]].head(3))

                                 Salary  \
0            486,49 - 1 801,80 € / mois   
1            486,49 - 1 801,80 € / mois   
2  Estimation → 12,36 - 13,50 € / heure   

                                         Description  \
0  Nous recherchons un·e candidat·e :  Alternance...   
1  Pourquoi nous rejoindre ?  > Participer à la t...   
2  Nous recherchons un(e) menuisier(e) expériment...   

                                   description_clean  \
0  Nous recherchons un·e candidat·e :  Alternance...   
1  Pourquoi nous rejoindre ?  > Participer à la t...   
2  Nous recherchons un(e) menuisier(e) expériment...   

                         salary_monthly  
0            486,49 - 1 801,80 € / mois  
1            486,49 - 1 801,80 € / mois  
2  Estimation → 12,36 - 13,50 € / heure  


## Étape 5 – Clustering thématique avec NMF

Pipeline :
1) TF-IDF sur `description_clean`
2) NMF (5 topics) pour extraire les thématiques
3) Attribution du cluster dominant dans `job_cluster`
4) Top mots par topic pour interprétation

In [4]:
# --- Cible binaire salaire ---
median_salary = df["salary_monthly"].median(skipna=True)
df["high_salary"] = (df["salary_monthly"] > median_salary).astype(int)

print(f"Médiane salaire mensuel: {median_salary:.2f}")
print(df["high_salary"].value_counts())

TypeError: Cannot convert ['486,49 - 1\u202f801,80 € / mois' '486,49 - 1\u202f801,80 € / mois'
 'Estimation → 12,36 - 13,50 € / heure' ... '11,88 - 12 € / heure'
 '11,89 € / heure' 'Pas de salaire renseigné'] to numeric

## Étape 6 – Classification salaire (Logistic Regression)

- Données d’entrée : TF-IDF sur `description_clean`
- Cible : `high_salary`
- Métriques : rapport de classification + AUC ROC

In [5]:
# --- TF-IDF + NMF ---
text_col = "description_clean"
texts = df[text_col].fillna("").astype(str)

vect = TfidfVectorizer(max_features=500, min_df=2, max_df=0.9)
X_text = vect.fit_transform(texts)

nmf = NMF(n_components=5, random_state=42, max_iter=400)
W = nmf.fit_transform(X_text)

df["job_cluster"] = W.argmax(axis=1)
print("Clustering NMF terminé. Répartition des clusters:")
print(df["job_cluster"].value_counts().sort_index())

# Interprétation des topics
def print_topics(nmf_model, feature_names, n_top_words=10):
    for i, comp in enumerate(nmf_model.components_):
        terms = [feature_names[j] for j in comp.argsort()[-n_top_words:]]
        print(f"Topic {i}: {', '.join(terms)}")

print("\nTop mots par topic:")
print_topics(nmf, vect.get_feature_names_out())

Clustering NMF terminé. Répartition des clusters:
job_cluster
0    193
1    188
2     99
3    227
4    512
Name: count, dtype: int64

Top mots par topic:
Topic 0: pourrez, quelques, collège, lycée, ou, cours, niveau, vos, vous, élèves
Topic 1: aventure, déplacement, engagés, 14h, semaine, tes, tu, produits, ton, esallia
Topic 2: un, 18, salaire, diplômes, vos, 14, missions, ouihelp, vous, aide
Topic 3: intelligente, école, domicile, une, ou, mission, vous, activités, garde, enfants
Topic 4: au, clients, votre, gestion, une, dans, un, du, le, vous


### b) Entraînement du modèle

In [6]:
# --- Classification salaire ---
X = X_text  # TF-IDF matrix déjà calculée
# Recalculez y au cas où
median_salary = df["salary_monthly"].median(skipna=True)
y = (df["salary_monthly"] > median_salary).astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

clf = LogisticRegression(max_iter=500)
clf.fit(X_train, y_train)

preds = clf.predict(X_test)
probs = clf.predict_proba(X_test)[:, 1]
report = classification_report(y_test, preds, digits=3)
auc = roc_auc_score(y_test, probs)

print(report)
print(f"AUC: {auc:.4f}")

TypeError: Cannot convert ['486,49 - 1\u202f801,80 € / mois' '486,49 - 1\u202f801,80 € / mois'
 'Estimation → 12,36 - 13,50 € / heure' ... '11,88 - 12 € / heure'
 '11,89 € / heure' 'Pas de salaire renseigné'] to numeric

## Étape 7 – Attacher les prédictions

On génère les prédictions sur l’ensemble du dataset et on ajoute:
- `pred_high_salary` (classe prédite)
- `pred_proba_high_salary` (probabilité associée)

In [7]:
# --- Prédictions complètes ---
df["pred_high_salary"] = clf.predict(X)
df["pred_proba_high_salary"] = clf.predict_proba(X)[:, 1]

print(df[["salary_monthly", "high_salary", "pred_high_salary", "pred_proba_high_salary"]].head(5))

NameError: name 'clf' is not defined

## Étape 8 – Sauvegarder le dataset enrichi

On enregistre le résultat pour le dashboard (Phase 4) :
- Fichier: `data/enriched/hellowork_ml_enriched.csv`
- Contient: texte nettoyé, clusters, labels salaire, prédictions

In [ ]:
# --- Sauvegarde ---
output_path = Path("data/enriched/hellowork_ml_enriched.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(output_path, index=False, encoding="utf-8")

print("="*60)
print("✓ PHASE 3 COMPLETE")
print(f"Enriched dataset saved: {output_path}")
print(f"Rows: {len(df)}")
print("="*60)

✓ PHASE 3 COMPLETE
Enriched dataset saved: data\enriched\hellowork_ml_enriched.csv
Rows: 1219


: 